In [ ]:
#0722 訓練抓力okok
import cv2
import mediapipe as mp
import math

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

# Calculate distance between two landmarks
def calculate_distance(landmark1, landmark2):
    return math.sqrt((landmark1[0] - landmark2[0])**2 + (landmark1[1] - landmark2[1])**2)

cap = cv2.VideoCapture(0)

# Counter variables
left_counter = 0 
right_counter = 0

# Max distance variables
left_max_distance = 0
right_max_distance = 0

# Stage variables
left_stage = None
right_stage = None

# Setup MediaPipe instance
with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = hands.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Get coordinates
                landmark1 = [hand_landmarks.landmark[12].x, hand_landmarks.landmark[12].y]
                landmark2 = [hand_landmarks.landmark[0].x, hand_landmarks.landmark[0].y]
                
                # Calculate distance
                distance = calculate_distance(landmark1, landmark2)
                
                # Check if hand is left or right
                handedness = results.multi_handedness[results.multi_hand_landmarks.index(hand_landmarks)].classification[0].label
                
                # Update max distance, stage and counter
                if handedness == "Left":
                    left_max_distance = max(left_max_distance, distance)
                    if distance < left_max_distance / 2 and left_stage != "down":
                        left_stage = "down"
                        left_counter += 1
                    if distance > left_max_distance / 2:
                        left_stage = "up"
                    cv2.putText(image, f"Left Distance: {distance:.2f}", (10,30), 
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                elif handedness == "Right":
                    right_max_distance = max(right_max_distance, distance)
                    if distance < right_max_distance / 2 and right_stage != "down":
                        right_stage = "down"
                        right_counter += 1
                    if distance > right_max_distance / 2:
                        right_stage = "up"
                    cv2.putText(image, f"Right Distance: {distance:.2f}", (10,60), 
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                
                # Draw hand landmarks
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        
        # Visualize counter and max distance
        cv2.putText(image, f"Left Counter: {left_counter}", (10,90), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2,cv2.LINE_AA)
        cv2.putText(image,f"Right Counter: {right_counter}",(10 ,120),
                    cv2.FONT_HERSHEY_SIMPLEX ,1,(255 ,255 ,255) ,2 ,cv2.LINE_AA)
        cv2.putText(image,f"Left Max Distance: {left_max_distance:.2f}",(10 ,150),
                    cv2.FONT_HERSHEY_SIMPLEX ,1,(255 ,255 ,255) ,2 ,cv2.LINE_AA)
        cv2.putText(image,f"Right Max Distance: {right_max_distance:.2f}",(10 ,180),
                    cv2.FONT_HERSHEY_SIMPLEX ,1,(255 ,255 ,255) ,2 ,cv2.LINE_AA)
        
        cv2.imshow('MediaPipe Hands', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
